# Baseline Time Series Model for Tic Intensity Prediction

**Goal**: Predict the intensity of the next tic based on previous tic history

**Approach**: Sequence-based prediction using sliding window

## 1. Load and Clean Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, f1_score, precision_recall_curve, auc, classification_report
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Load data
df = pd.read_csv('results (2).csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Data cleaning
print("=" * 60)
print("DATA CLEANING")
print("=" * 60)

# Parse timestamps
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = pd.to_datetime(df['date'])

# Convert intensity to numeric
df['intensity'] = pd.to_numeric(df['intensity'], errors='coerce')

# Handle 'null' strings
for col in ['mood', 'trigger']:
    df[col] = df[col].replace('null', np.nan)

# Sort by user and timestamp (CRITICAL for time series)
df = df.sort_values(['userId', 'timestamp']).reset_index(drop=True)

print(f"\nTotal records: {len(df)}")
print(f"Unique users: {df['userId'].nunique()}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"\nMissing values:")
print(df[['intensity', 'type', 'timeOfDay', 'mood', 'trigger']].isnull().sum())

In [ ]:
# Quick EDA on intensity
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Intensity distribution
axes[0].hist(df['intensity'].dropna(), bins=10, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Intensity')
axes[0].set_ylabel('Count')
axes[0].set_title('Intensity Distribution')
axes[0].axvline(df['intensity'].mean(), color='red', linestyle='--', label=f'Mean: {df["intensity"].mean():.2f}')
axes[0].legend()

# High vs low intensity
df['high_intensity'] = (df['intensity'] >= 7).astype(int)
high_count = df['high_intensity'].sum()
low_count = len(df) - high_count
axes[1].bar(['Low (<7)', 'High (≥7)'], [low_count, high_count], alpha=0.7, edgecolor='black')
axes[1].set_ylabel('Count')
axes[1].set_title(f'Binary Classification Target\n(High: {high_count/len(df)*100:.1f}%)')

plt.tight_layout()
plt.show()

print(f"\nIntensity Statistics:")
print(f"  Mean: {df['intensity'].mean():.2f}")
print(f"  Median: {df['intensity'].median():.1f}")
print(f"  Std: {df['intensity'].std():.2f}")
print(f"  High-intensity rate: {high_count/len(df)*100:.1f}%")

## 2. Feature Engineering

In [ ]:
print("=" * 60)
print("FEATURE ENGINEERING")
print("=" * 60)

# Extract temporal features
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek  # 0=Monday, 6=Sunday
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)

# Calculate time since previous tic (per user)
df['time_since_prev_hours'] = df.groupby('userId')['timestamp'].diff().dt.total_seconds() / 3600

# Calculate user-level statistics (up to current tic)
df['user_mean_intensity'] = df.groupby('userId')['intensity'].expanding().mean().shift(1).reset_index(level=0, drop=True)
df['user_std_intensity'] = df.groupby('userId')['intensity'].expanding().std().shift(1).reset_index(level=0, drop=True)
df['user_tic_count'] = df.groupby('userId').cumcount()  # Number of tics so far

# Lag features (previous tic intensity)
df['prev_intensity_1'] = df.groupby('userId')['intensity'].shift(1)
df['prev_intensity_2'] = df.groupby('userId')['intensity'].shift(2)
df['prev_intensity_3'] = df.groupby('userId')['intensity'].shift(3)

# Previous tic type
df['prev_type_1'] = df.groupby('userId')['type'].shift(1)

# Encode categorical features
le_type = LabelEncoder()
le_prev_type = LabelEncoder()
le_time_of_day = LabelEncoder()

df['type_encoded'] = le_type.fit_transform(df['type'].fillna('unknown'))
df['prev_type_encoded'] = le_prev_type.fit_transform(df['prev_type_1'].fillna('unknown'))
df['timeOfDay_encoded'] = le_time_of_day.fit_transform(df['timeOfDay'].fillna('unknown'))

# Mood and trigger (optional - only when available)
df['has_mood'] = (~df['mood'].isna()).astype(int)
df['has_trigger'] = (~df['trigger'].isna()).astype(int)

# Encode mood and trigger with 'missing' category
le_mood = LabelEncoder()
le_trigger = LabelEncoder()
df['mood_encoded'] = le_mood.fit_transform(df['mood'].fillna('missing'))
df['trigger_encoded'] = le_trigger.fit_transform(df['trigger'].fillna('missing'))

print("\nFeatures created:")
print("  - Temporal: hour, day_of_week, is_weekend")
print("  - Sequence: prev_intensity_1/2/3, time_since_prev_hours")
print("  - User-level: user_mean_intensity, user_std_intensity, user_tic_count")
print("  - Categorical: type, timeOfDay, prev_type (encoded)")
print("  - Optional: mood, trigger (when available)")

df.head(10)

## 3. Create Training Sequences

In [ ]:
print("=" * 60)
print("CREATING TRAINING SEQUENCES")
print("=" * 60)

# Select features for modeling
feature_cols = [
    # Temporal
    'hour', 'day_of_week', 'is_weekend', 'timeOfDay_encoded',
    # Sequence features
    'prev_intensity_1', 'prev_intensity_2', 'prev_intensity_3',
    'time_since_prev_hours',
    # Type features
    'type_encoded', 'prev_type_encoded',
    # User-level
    'user_mean_intensity', 'user_std_intensity', 'user_tic_count',
    # Optional
    'mood_encoded', 'trigger_encoded', 'has_mood', 'has_trigger'
]

# Target variables
target_regression = 'intensity'
target_classification = 'high_intensity'

# Filter: only keep rows where we have at least 3 previous tics (for sequence)
# This means dropping first 3 tics per user
df_model = df[df['user_tic_count'] >= 3].copy()

# Remove rows with missing target or critical features
df_model = df_model.dropna(subset=[target_regression] + ['prev_intensity_1', 'prev_intensity_2', 'prev_intensity_3'])

# Fill remaining missing values
df_model['time_since_prev_hours'] = df_model['time_since_prev_hours'].fillna(df_model['time_since_prev_hours'].median())
df_model['user_std_intensity'] = df_model['user_std_intensity'].fillna(0)

print(f"\nOriginal dataset: {len(df)} records")
print(f"After filtering (≥3 prev tics): {len(df_model)} records")
print(f"Users in modeling dataset: {df_model['userId'].nunique()}")
print(f"\nHigh-intensity rate in model data: {df_model['high_intensity'].mean()*100:.1f}%")

In [ ]:
# User-grouped train/test split (80/20)
# Important: split by user to avoid data leakage
users = df_model['userId'].unique()
train_users, test_users = train_test_split(users, test_size=0.2, random_state=42)

train_df = df_model[df_model['userId'].isin(train_users)]
test_df = df_model[df_model['userId'].isin(test_users)]

# Also create temporal split within each user (alternative evaluation)
# Use first 80% of each user's tics for training
def temporal_split(group, train_frac=0.8):
    n_train = int(len(group) * train_frac)
    group['temporal_split'] = ['train'] * n_train + ['test'] * (len(group) - n_train)
    return group

df_model = df_model.groupby('userId', group_keys=False).apply(temporal_split)
train_temporal = df_model[df_model['temporal_split'] == 'train']
test_temporal = df_model[df_model['temporal_split'] == 'test']

print(f"\nUser-grouped split:")
print(f"  Train: {len(train_df)} samples, {len(train_users)} users")
print(f"  Test: {len(test_df)} samples, {len(test_users)} users")

print(f"\nTemporal split (within-user):")
print(f"  Train: {len(train_temporal)} samples")
print(f"  Test: {len(test_temporal)} samples")

# Prepare features and targets
X_train = train_df[feature_cols]
y_train_reg = train_df[target_regression]
y_train_clf = train_df[target_classification]

X_test = test_df[feature_cols]
y_test_reg = test_df[target_regression]
y_test_clf = test_df[target_classification]

print(f"\nFeature matrix shape: {X_train.shape}")
print(f"\nFeatures used: {feature_cols}")

## 4. Train Baseline Models

### 4.1 Regression Model (Predict Exact Intensity 1-10)

In [ ]:
print("=" * 60)
print("REGRESSION MODEL: Predict Intensity (1-10)")
print("=" * 60)

# Train Random Forest Regressor
rf_reg = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf_reg.fit(X_train, y_train_reg)

# Predictions
y_pred_train = rf_reg.predict(X_train)
y_pred_test = rf_reg.predict(X_test)

# Evaluation
train_mae = mean_absolute_error(y_train_reg, y_pred_train)
test_mae = mean_absolute_error(y_test_reg, y_pred_test)
train_rmse = np.sqrt(mean_squared_error(y_train_reg, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test_reg, y_pred_test))

print(f"\nRandom Forest Regressor Results:")
print(f"  Train MAE: {train_mae:.3f}")
print(f"  Test MAE:  {test_mae:.3f}")
print(f"  Train RMSE: {train_rmse:.3f}")
print(f"  Test RMSE:  {test_rmse:.3f}")

# Baseline comparison (predict mean intensity)
baseline_pred = np.full(len(y_test_reg), y_train_reg.mean())
baseline_mae = mean_absolute_error(y_test_reg, baseline_pred)
baseline_rmse = np.sqrt(mean_squared_error(y_test_reg, baseline_pred))

print(f"\nBaseline (predict mean) Results:")
print(f"  Baseline MAE: {baseline_mae:.3f}")
print(f"  Baseline RMSE: {baseline_rmse:.3f}")
print(f"\nImprovement over baseline:")
print(f"  MAE improvement: {(baseline_mae - test_mae) / baseline_mae * 100:.1f}%")
print(f"  RMSE improvement: {(baseline_rmse - test_rmse) / baseline_rmse * 100:.1f}%")

In [ ]:
# Visualize predictions vs actuals
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
axes[0].scatter(y_test_reg, y_pred_test, alpha=0.5, s=20)
axes[0].plot([1, 10], [1, 10], 'r--', label='Perfect prediction')
axes[0].set_xlabel('Actual Intensity')
axes[0].set_ylabel('Predicted Intensity')
axes[0].set_title(f'Regression: Predicted vs Actual\n(Test MAE: {test_mae:.3f})')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residuals
residuals = y_test_reg - y_pred_test
axes[1].hist(residuals, bins=30, edgecolor='black', alpha=0.7)
axes[1].axvline(0, color='red', linestyle='--', label='Zero error')
axes[1].set_xlabel('Prediction Error (Actual - Predicted)')
axes[1].set_ylabel('Count')
axes[1].set_title('Residual Distribution')
axes[1].legend()

plt.tight_layout()
plt.show()

### 4.2 Binary Classification Model (High ≥7 vs Low <7)

In [ ]:
print("=" * 60)
print("BINARY CLASSIFICATION: High (≥7) vs Low (<7)")
print("=" * 60)

# Train Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf_clf.fit(X_train, y_train_clf)

# Predictions
y_pred_clf = rf_clf.predict(X_test)
y_pred_proba = rf_clf.predict_proba(X_test)[:, 1]  # Probability of high intensity

# Evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(y_test_clf, y_pred_clf)
precision = precision_score(y_test_clf, y_pred_clf)
recall = recall_score(y_test_clf, y_pred_clf)
f1 = f1_score(y_test_clf, y_pred_clf)

print(f"\nRandom Forest Classifier Results:")
print(f"  Accuracy:  {accuracy:.3f}")
print(f"  Precision: {precision:.3f}")
print(f"  Recall:    {recall:.3f}")
print(f"  F1-score:  {f1:.3f}")

# Calculate PR-AUC (important for imbalanced data)
precision_curve, recall_curve, _ = precision_recall_curve(y_test_clf, y_pred_proba)
pr_auc = auc(recall_curve, precision_curve)
print(f"  PR-AUC:    {pr_auc:.3f}")

# Baseline (always predict majority class)
baseline_acc = (y_test_clf == 0).mean()  # Accuracy if we always predict "low"
print(f"\nBaseline (predict majority class):")
print(f"  Baseline Accuracy: {baseline_acc:.3f}")
print(f"  Model improvement: {(accuracy - baseline_acc) / baseline_acc * 100:.1f}%")

print(f"\nClassification Report:")
print(classification_report(y_test_clf, y_pred_clf, target_names=['Low (<7)', 'High (≥7)']))

In [ ]:
# Visualize classification results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Precision-Recall curve
axes[0].plot(recall_curve, precision_curve, label=f'PR-AUC = {pr_auc:.3f}')
axes[0].set_xlabel('Recall')
axes[0].set_ylabel('Precision')
axes[0].set_title('Precision-Recall Curve')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test_clf, y_pred_clf)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1],
            xticklabels=['Low', 'High'], yticklabels=['Low', 'High'])
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')
axes[1].set_title('Confusion Matrix')

plt.tight_layout()
plt.show()

## 5. Feature Importance Analysis

In [ ]:
print("=" * 60)
print("FEATURE IMPORTANCE ANALYSIS")
print("=" * 60)

# Get feature importances
feature_importance_reg = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_reg.feature_importances_
}).sort_values('importance', ascending=False)

feature_importance_clf = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_clf.feature_importances_
}).sort_values('importance', ascending=False)

# Plot top 15 features
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Regression
top_features_reg = feature_importance_reg.head(15)
axes[0].barh(range(len(top_features_reg)), top_features_reg['importance'], alpha=0.7)
axes[0].set_yticks(range(len(top_features_reg)))
axes[0].set_yticklabels(top_features_reg['feature'])
axes[0].set_xlabel('Importance')
axes[0].set_title('Top 15 Features - Regression Model')
axes[0].invert_yaxis()

# Classification
top_features_clf = feature_importance_clf.head(15)
axes[1].barh(range(len(top_features_clf)), top_features_clf['importance'], alpha=0.7, color='orange')
axes[1].set_yticks(range(len(top_features_clf)))
axes[1].set_yticklabels(top_features_clf['feature'])
axes[1].set_xlabel('Importance')
axes[1].set_title('Top 15 Features - Classification Model')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

print("\nTop 10 Features - Regression:")
print(feature_importance_reg.head(10).to_string(index=False))

print("\nTop 10 Features - Classification:")
print(feature_importance_clf.head(10).to_string(index=False))

## 6. Per-User Performance Analysis

In [ ]:
print("=" * 60)
print("PER-USER PERFORMANCE ANALYSIS")
print("=" * 60)

# Calculate MAE per user in test set
test_df_with_pred = test_df.copy()
test_df_with_pred['pred_intensity'] = y_pred_test
test_df_with_pred['abs_error'] = np.abs(test_df_with_pred['intensity'] - test_df_with_pred['pred_intensity'])

user_performance = test_df_with_pred.groupby('userId').agg({
    'abs_error': 'mean',
    'intensity': ['count', 'mean', 'std']
}).round(3)

user_performance.columns = ['mae', 'n_samples', 'mean_intensity', 'std_intensity']
user_performance = user_performance.sort_values('mae')

print(f"\nPer-user MAE statistics:")
print(f"  Mean MAE across users: {user_performance['mae'].mean():.3f}")
print(f"  Median MAE: {user_performance['mae'].median():.3f}")
print(f"  Best user MAE: {user_performance['mae'].min():.3f}")
print(f"  Worst user MAE: {user_performance['mae'].max():.3f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# MAE distribution across users
axes[0].hist(user_performance['mae'], bins=15, edgecolor='black', alpha=0.7)
axes[0].axvline(user_performance['mae'].mean(), color='red', linestyle='--', label='Mean')
axes[0].set_xlabel('MAE per User')
axes[0].set_ylabel('Number of Users')
axes[0].set_title('Distribution of MAE Across Users')
axes[0].legend()

# MAE vs number of samples
axes[1].scatter(user_performance['n_samples'], user_performance['mae'], alpha=0.6)
axes[1].set_xlabel('Number of Test Samples per User')
axes[1].set_ylabel('MAE')
axes[1].set_title('MAE vs Sample Size\n(Do we perform better with more data?)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nTop 5 best-predicted users:")
print(user_performance.head(5))

print(f"\nTop 5 worst-predicted users:")
print(user_performance.tail(5))

## 7. Summary and Next Steps

In [ ]:
print("=" * 60)
print("BASELINE MODEL SUMMARY")
print("=" * 60)

print(f"\n📊 Dataset:")
print(f"   - Total usable sequences: {len(df_model)}")
print(f"   - Users: {df_model['userId'].nunique()}")
print(f"   - Features: {len(feature_cols)}")

print(f"\n🎯 Regression Performance (Predict Intensity 1-10):")
print(f"   - Test MAE: {test_mae:.3f}")
print(f"   - Test RMSE: {test_rmse:.3f}")
print(f"   - Improvement over baseline: {(baseline_mae - test_mae) / baseline_mae * 100:.1f}%")

print(f"\n🎯 Classification Performance (High ≥7 vs Low <7):")
print(f"   - Test Accuracy: {accuracy:.3f}")
print(f"   - F1-score: {f1:.3f}")
print(f"   - PR-AUC: {pr_auc:.3f}")
print(f"   - Precision: {precision:.3f}")
print(f"   - Recall: {recall:.3f}")

print(f"\n🔑 Top 3 Most Important Features (Regression):")
for i, row in feature_importance_reg.head(3).iterrows():
    print(f"   {i+1}. {row['feature']}: {row['importance']:.4f}")

print(f"\n💡 Next Steps:")
print(f"   1. Try XGBoost for potentially better performance")
print(f"   2. Experiment with deeper sequence models (LSTM)")
print(f"   3. Build user-specific models for high-engagement users")
print(f"   4. Implement cold-start strategy for new users")
print(f"   5. Test different prediction windows (3-day, 7-day aggregates)")
print(f"   6. Add more feature engineering (interaction terms, polynomial features)")
print(f"   7. Hyperparameter tuning with grid search")